In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams["font.family"] = "Hiragino Maru Gothic Pro"
import warnings
warnings.filterwarnings("ignore")

In [55]:
import requests
import io

url = 'https://covid19.mhlw.go.jp/public/opendata/newly_confirmed_cases_daily.csv'
res = requests.get(url).content
df = pd.read_csv(io.StringIO(res.decode('utf-8')), header=0, index_col=0, parse_dates=True)


df["log_Tokyo"] = np.log1p(df[target_column]).diff()
# 前日
df["a_day_ago_Tokyo"] = df["log_Tokyo"].shift(1)

In [56]:
df.head()

,ALL,Hokkaido,Aomori,Iwate,Miyagi,Akita,Yamagata,Fukushima,Ibaraki,Tochigi,...,Fukuoka,Saga,Nagasaki,Kumamoto,Oita,Miyazaki,Kagoshima,Okinawa,log_Tokyo,a_day_ago_Tokyo
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-16,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
2020-01-17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,NaN
2020-01-18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
2020-01-19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0
2020-01-20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0,0.0


In [57]:
df.tail()

,ALL,Hokkaido,Aomori,Iwate,Miyagi,Akita,Yamagata,Fukushima,Ibaraki,Tochigi,...,Fukuoka,Saga,Nagasaki,Kumamoto,Oita,Miyazaki,Kagoshima,Okinawa,log_Tokyo,a_day_ago_Tokyo
Date,,,,,,,,,,,,,,,,,,,,,
2022-05-03,30433,1718,296,212,389,310,113,346,407,499,...,1856,294,412,567,391,357,704,1761,0.334212,-0.274071
2022-05-04,26421,2090,246,221,326,173,137,319,498,207,...,1557,175,361,314,247,372,469,1201,-0.112733,0.334212
2022-05-05,20713,1854,290,197,210,170,125,339,481,233,...,1372,244,229,255,230,225,420,1327,-0.256614,-0.112733
2022-05-06,21547,1668,153,181,230,181,115,324,340,262,...,1181,294,203,290,258,287,521,1398,0.144565,-0.256614
2022-05-07,39239,2112,356,270,538,329,201,499,348,638,...,2209,495,578,736,466,464,928,2375,0.351066,0.144565


In [58]:
# 目的変数
target_column = "log_Tokyo"
# 説明変数 
ex_columns = [
    "a_day_ago_Tokyo",
]

# 学習期間の設定
train_start = df.index.get_loc("2022-01-01")
train_end = df.index.get_loc("2022-04-07")
test_start = df.index.get_loc("2022-04-07")
test_end = df.index.get_loc("2022-05-07")

w = 7 # 窓幅
N = 1 # 予測先

# 学習データ
df_tr = df.iloc[train_start : train_end]
Ytr = df_tr[target_column].to_numpy().ravel()
Xtr = df_tr[target_column].to_numpy()
Ytr_org = df.iloc[train_start-1 : train_end]["Tokyo"]
Ytr_org = Ytr_org[N+w-1 : -1]
print("Training data", len(Xtr))


# テストデータ
df_ts = df.iloc[test_start : test_end]
Yts = df_ts[target_column].to_numpy().ravel()
Xts = df_ts[target_column].to_numpy()
Yts_org = df.iloc[test_start-1 : test_end]["Tokyo"]
Yts_org = Yts_org[N+w-1 : -1]
print("Test data", len(Xts))

Training data 96
Test data 30


In [60]:
Ytr.shape, Yts.shape

((96,), (30,))

In [79]:
from sklearn.preprocessing import StandardScaler

# 標準化
scalerX = StandardScaler()
scalerY = StandardScaler()
Xtr_norm = scalerX.fit_transform(Xtr.reshape(-1, 1))
Ytr_norm = scalerY.fit_transform(Ytr.reshape(-1, 1))
Xts_norm = scalerX.transform(Xts.reshape(-1, 1))
Yts_norm = scalerY.transform(Yts.reshape(-1, 1))

In [80]:
Xtr_norm.shape

(96, 1)

In [81]:
Xtr[1:]

array([-2.76253377e-01,  6.85542581e-01,  1.48683198e-01,  1.04222757e+00,
        4.97747436e-01,  3.54839692e-01,  3.19082973e-01, -7.15122178e-02,
       -2.80756304e-01, -6.11360000e-02,  9.67149149e-01,  4.58052331e-01,
        1.24440272e-01,  1.89488673e-01, -1.75291014e-02, -5.03275496e-01,
        7.95126867e-01,  2.68485938e-01,  1.43056457e-01,  1.82022465e-02,
        1.64391761e-01, -1.55085871e-01, -4.61061692e-01,  8.61085376e-01,
       -2.02091069e-02,  5.81143400e-02,  5.41784466e-02,  7.35848784e-02,
       -1.65041599e-01, -5.41372065e-01,  7.06201079e-01, -1.42932226e-02,
        2.19179551e-02,  4.48186391e-02,  4.88039005e-03, -2.41390223e-01,
       -5.44901246e-01,  7.92096709e-01, -1.04252237e-01,  2.09357129e-02,
       -1.30019985e-01, -7.10593106e-01,  5.90227987e-01, -6.85098289e-01,
        9.03297213e-01, -8.34230084e-02,  8.51996157e-04, -1.85111503e-01,
        3.58043567e-03, -2.24238242e-01, -6.47299501e-01,  8.98380646e-01,
        2.25375496e-02, -

In [88]:
# 学習データ
Xtr_trs = Xtr_norm
for i in range(w):
    Xtr_trs = np.hstack([Xtr_trs, np.roll(Xtr_norm, i+1)])
Xtr_trs = Xtr_trs[N+w-1:-N]
Ytr_trs = np.roll(Ytr_norm, -N)[:-N]
Ytr_trs = Ytr_trs[N+w-1:]
print(Xtr_trs.shape, Ytr_trs.shape)


# テストデータ
Xts_trs = Xts_norm
for i in range(w):
    Xts_trs = np.hstack([Xts_trs, np.roll(Xts_norm, i+1)])
Xts_trs = Xts_trs[N+w-1:-N]
Yts_trs = np.roll(Yts_norm, -N)[:-N]
Yts_trs = Yts_trs[N+w-1:]
print(Xts_trs.shape, Yts_trs.shape)

(88, 8) (88, 1)
(22, 8) (22, 1)


In [86]:
Xtr_trs

array([[ 0.59433456,  0.6732435 ,  0.98861637,  2.1901906 ,  0.21829149,
         1.40304803, -0.71947062,  0.13115728],
       [-0.26764201,  0.59433456,  0.6732435 ,  0.98861637,  2.1901906 ,
         0.21829149,  1.40304803, -0.71947062],
       [-0.72940781, -0.26764201,  0.59433456,  0.6732435 ,  0.98861637,
         2.1901906 ,  0.21829149,  1.40304803],
       [-0.24474348, -0.72940781, -0.26764201,  0.59433456,  0.6732435 ,
         0.98861637,  2.1901906 ,  0.21829149],
       [ 2.02450541, -0.24474348, -0.72940781, -0.26764201,  0.59433456,
         0.6732435 ,  0.98861637,  2.1901906 ],
       [ 0.90101608,  2.02450541, -0.24474348, -0.72940781, -0.26764201,
         0.59433456,  0.6732435 ,  0.98861637],
       [ 0.16479151,  0.90101608,  2.02450541, -0.24474348, -0.72940781,
        -0.26764201,  0.59433456,  0.6732435 ],
       [ 0.30834217,  0.16479151,  0.90101608,  2.02450541, -0.24474348,
        -0.72940781, -0.26764201,  0.59433456],
       [-0.14851054,  0.30834217

In [87]:
Ytr_trs

array([[-0.26764201],
       [-0.72940781],
       [-0.24474348],
       [ 2.02450541],
       [ 0.90101608],
       [ 0.16479151],
       [ 0.30834217],
       [-0.14851054],
       [-1.22046948],
       [ 1.64488175],
       [ 0.48267558],
       [ 0.20587424],
       [-0.06965758],
       [ 0.25295759],
       [-0.45207473],
       [-1.12731086],
       [ 1.79044086],
       [-0.15442485],
       [ 0.01842156],
       [ 0.00973572],
       [ 0.05256238],
       [-0.47404531],
       [-1.30454208],
       [ 1.44863779],
       [-0.14136951],
       [-0.06145765],
       [-0.01091977],
       [-0.09905663],
       [-0.64253362],
       [-1.31233037],
       [ 1.63819471],
       [-0.33989362],
       [-0.06362529],
       [-0.39675861],
       [-1.67798388],
       [ 1.19270508],
       [-1.62172119],
       [ 1.88359514],
       [-0.29392708],
       [-0.10794661],
       [-0.51833614],
       [-0.10192541],
       [-0.60468214],
       [-1.53830575],
       [ 1.87274512],
       [-0